# Proceso de Automatización para ANY Manufacturing

Configuración de la conexión a la base de datos.

Se definen las credenciales para la empresa de la base de datos (servidor, nombre de la base de datos, usuario y contraseña).
Se construye una cadena de conexión (connection_string) usando sqlalchemy.create_engine, que permite conectar y ejecutar consultas SQL en la base de datos.

In [ ]:
database_config = {
    'server':'servidor',
    'database': 'Base de datos para la empresa',
    'username': 'Usuario',
    'password': 'Contraseña',
    'driver': '{ODBC Driver 17 for SQL Server}'
}
connection_string = f"mssql+pyodbc://{database_config['username']}:{database_config['password']}@{database_config['server']}/{database_config['database']}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)


Clase DataHandler para manejar eventos de archivos: Esta clase personalizada, DataHandler, hereda de FileSystemEventHandler de watchdog, y responde a eventos de creación y modificación de archivos en una carpeta específica.

In [ ]:
class DataHandler(FileSystemEventHandler):
    def __init__(self, engine):
        self.engine = engine


Procesamiento de nuevos archivos: Los métodos on_created y on_modified verifican si el archivo es un .csv, y si es así, llaman a process_new_file, donde se carga el archivo y se preparan los datos para su inserción en la base de datos.

In [ ]:
def on_created(self, event):
    if event.is_directory or not event.src_path.endswith(".csv"):
        return
    self.process_new_file(event.src_path)


Inserción de datos en SQL: En process_new_file, el archivo .csv se carga en un DataFrame, convirtiendo columnas con fechas y eliminando columnas no deseadas como Unnamed: 0. Luego, la tabla SQL correspondiente y la columna ID se determinan mediante get_id_column, y se identifican los registros nuevos que no están ya en la tabla.

In [ ]:
def process_new_file(self, file_path):
    new_data = pd.read_csv(file_path)
    new_data = new_data.drop(columns=['Unnamed: 0'], errors='ignore')
    for column in new_data.columns:
        if 'date' in column.lower():
            new_data[column] = pd.to_datetime(new_data[column], errors='coerce')
    # Identificación de filas nuevas e inserción
    new_rows = new_data[~new_data[id_column].isin(existing_data[id_column])]
    new_rows.to_sql(table_name, self.engine, if_exists='append', index=False)


Monitorización continua de archivos: En el bloque principal (__main__), Observer monitoriza una carpeta específica, path_to_watch. Los archivos existentes en esta carpeta se procesan inicialmente con process_existing_files.

In [ ]:
path_to_watch = r"C:\Users\Admin\Desktop\proyecto final\Proceso\CSV listos"
observer = Observer()
observer.schedule(event_handler, path=path_to_watch, recursive=True)
observer.start()


Este flujo automatiza el proceso de carga y actualización de datos en SQL Server, optimizando la integración y gestión de archivos .csv en tiempo real.